## Perform predictions using the trained model

This notebook contains the code to carry out predictions with the trained
models. Note that it is currently set up only to carry out segmentations
(predictions) for the left out images in the case of the LOIO experiments, and
for the images defined in the `test_images` list in the configuration file for
the *final* model. We briefly summarise the prediction process. For a given
test image and its hand-drawn axis of maximum growth image, predictions are
generated as follows:

1. The growth axis is extracted from the given growth axis image.
2. Patches the same size as those used to train the model are extracted along
    the growth axis, with a user-defined stride
    (we use 16 pixels in this work).
3. The model predicts the segmentation mask of each extracted patch.
4. Predictions are saved to the same folder the model is stored in.

Please see the project [README](./README.md) for full details of the prediction
procedure.

In [1]:
# We need the imagecodecs package for imageio (used in a library) to
# parse .tif files -- this can be skipped if no .tif files are used.
%pip install imagecodecs

     |████████████████████████████████| 31.0 MB 1.2 MB/s 


In [2]:
import os
import google
import yaml
import tqdm.notebook
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# SETTINGS
google_drive_path = r"/content/gdrive/MyDrive/2021-07_Rings_Project"

# auto-reload modules
%load_ext autoreload
%autoreload 2

In [4]:
# mount Google drive (local/home version)
google.colab.drive.mount("/content/gdrive")

# change to the base directory that contains the setting file and where
# we wish to save the trained models to
%cd {google_drive_path}

# load the config file
with open("config.yaml", "r") as fd:
    cfg = yaml.safe_load(fd)
print('Config file loaded')

Mounted at /content/gdrive
/content/gdrive/MyDrive/2021-07_Rings_Project
Config file loaded


In [5]:
# import our needed functions. Note that these are hosted on the google drive,
# so we have to connect to it first.
from shellai import preprocessing, tf_util

In [6]:
# connect to the TPU -- this greatly speeds up predictions
print("OS Version & Details: ")
!lsb_release -a
print()

tpu_device_location = f"grpc://{os.environ['COLAB_TPU_ADDR']}"
print(f"TPU is allocated successfully at location: {tpu_device_location}.")
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_device_location)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy()

OS Version & Details: 
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic

TPU is allocated successfully at location: grpc://10.37.56.18:8470.
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.37.56.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.56.18:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Leave-one-image-out predictions



In [ ]:
# experimental settings

# list of image names. For each image, we train a model on all other images and
# test on the left out image
test_images = cfg['images']['train_images']

# size of patches to extract for prediction
# (this should be the same as the model was trained on)
patch_shape = tuple(cfg['testing']['patch_shape'])

# stride (in pixels) between extracted patches
stride_step = cfg['testing']['stride']

# batch size to push through the model (too large causes memory issues)
batch_size = cfg['testing']['batch_size']

# directory containing directories of training images
base_dir = cfg['images']['train_dir']

# directory containing directories of saved models. note that we will also save
# predictions to the corresponding model's directory
saved_model_dir = "saved_models"

In [ ]:
# perform predictions for each model (i.e predict each test image)
for test_image in tqdm.notebook.tqdm(test_images, leave=False):
    # directory where the models are saved
    model_dir = f"{saved_model_dir}/{test_image}"

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # PLOT THE OPTIMISATION HISTORY
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    history_path = f"{model_dir}/history.npz"

    with np.load(history_path, allow_pickle=True) as fd:
        history = fd['history'].item()

    keys = [key for key in history if "val" not in key]

    start = 0
    n = len(history[keys[0]])

    fig, axes = plt.subplots(1, len(keys), figsize=(15, 5), sharex=True)
    axes = axes if len(keys) > 1 else [axes]
    for ax, key in zip(axes, keys):
        ax.plot(
            range(1 + start, n + 1),
            history[key][start:],
            label="Training",
            alpha=0.5,
        )
        ax.plot(
            range(1 + start, n + 1),
            history[f"val_{key:s}"][start:],
            label="Validation",
            alpha=0.5,
        )
        ax.legend(loc="best")
        ax.set_title(key)

        ax.semilogy()
        ax.set_xlabel("epochs")

        for xl in range(5, n+5, 5):
            ax.axvline(x=xl, ls='-', c='k', alpha=0.1)

    plt.tight_layout()

    savepath = os.path.join(model_dir, "history.png")
    # plt.savefig(savepath, bbox_inchest='tight')
    plt.close()
    print(f'History plot saved to: {savepath}')    

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # PERFORM PREDICTIONS
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
 
    # load the image and image of the drawn line (not binary!)
    image, line_mask, _ = preprocessing.load_image_data(
        base_dir, test_image
    )

    # get the coords of the test patches (patch_coords), their centres (lc), 
    # and the coordinates of the hand-drawn axis of maximum growth (full_lc)
    patch_coords, lc, full_lc = tf_util.get_test_patch_locations(
        line_mask, patch_shape, stride_step
    )

    # create the test dataset
    with strategy.scope():
        test_ds = tf_util.convert_patches_into_tf_ds(
            patch_coords, image, batch_size=batch_size
        )
    
    # get the patch to each saved model (we save one ever N iterations)
    model_paths = tf.io.gfile.glob(
        os.path.join(model_dir, 'model_checkpoint_*.h5')
    )

    # for each model, perform the predictions
    for model_path in tqdm.notebook.tqdm(model_paths, leave=False):
        model_name = os.path.basename(model_path.split('.')[0])
        save_path = os.path.join(model_dir, f"{model_name:s}_predictions.npz")

        # if the predictions already exist, skip them
        if os.path.exists(save_path):
            print(f'Skipping: {save_path}')
            continue

        # otherwise, load the model on the TPU
        with strategy.scope():
            model = tf.keras.models.load_model(model_path)

        # predict all the images of the test dataset (batched)
        predictions = []

        for batch in test_ds:
            # make the prediction
            pred = model(batch, training=False)

            # convert to numpy and store
            predictions.append(pred.numpy().squeeze())

        # join together the predictions
        predictions = np.concatenate(predictions, axis=0)
        
        # finally, save them
        np.savez(
            save_path,
            predictions=predictions,
            full_line_coords=full_lc,
            patch_centres=lc,
            patch_coords=patch_coords
        )

        print('Saved:', save_path)

### Final model predictions

In [7]:
# experimental settings

# test images -- these have not been seen by the final model before
unseen_test_images = cfg['images']['test_images']

# size of patches to extract for prediction
# (this should be the same as the model was trained on)
patch_shape = tuple(cfg['testing']['patch_shape'])

# stride (in pixels) between extracted patches
stride_step = cfg['testing']['stride']

# batch size to push through the model (too large causes memory issues)
batch_size = cfg['testing']['batch_size']

# directory containing directories of images
base_dir = cfg['images']['test_dir']

# directory containing directories of saved models. note that we will also save
# predictions to the corresponding model's directory
saved_model_dir = "saved_models"

# final model name (this is the directory name stored in the dir above)
final_model_name = "final_model"

In [ ]:
# Note: - we only have one model because we trained on all training images.
#       - no history is avaliable because we have no validation data
model_dir = f"{saved_model_dir}/{final_model_name}"
model_paths = tf.io.gfile.glob(os.path.join(model_dir, 'model_checkpoint_*.h5'))

for image_name in tqdm.notebook.tqdm(unseen_test_images):
    # load the image and image of the drawn line (not binary!)
    image, line_mask = preprocessing.load_image_data(
        base_dir, image_name, no_rings=True
    )

    # get the coords of the test patches (patch_coords), their centres (lc), 
    # and the coordinates of the hand-drawn axis of maximum growth (full_lc)
    patch_coords, lc, full_lc = tf_util.get_test_patch_locations(
        line_mask, patch_shape, stride_step
    )

    # create the test dataset
    with strategy.scope():
        test_ds = tf_util.convert_patches_into_tf_ds(
            patch_coords, image, batch_size=batch_size
        )

    # for each model, perform the predictions
    for model_path in tqdm.notebook.tqdm(model_paths, leave=False):
        model_name = os.path.basename(model_path.split('.')[0])
        save_path = os.path.join(model_dir, f"{model_name:s}_predictions_{image_name:s}.npz")

        # if the predictions already exist, skip them
        if os.path.exists(save_path):
            print(f'Skipping: {save_path}')
            continue

        # otherwise, load the model on the TPU
        with strategy.scope():
            model = tf.keras.models.load_model(model_path)

        # predict all the images of the test dataset (batched)
        predictions = []

        for batch in test_ds:
            # make the prediction
            pred = model(batch, training=False)

            # convert to numpy and store
            predictions.append(pred.numpy().squeeze())

        # join together the predictions
        predictions = np.concatenate(predictions, axis=0)

        # finally, save them
        np.savez(
            save_path,
            predictions=predictions,
            full_line_coords=full_lc,
            patch_centres=lc,
            patch_coords=patch_coords
        )

        print('Saved:', save_path)